In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
data = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
warnings.filterwarnings('ignore') #Avoid print warnings

When we are in front of a new data set the first thing to do is view the amount of data available, and see if we are in fornt of a classification or regression model.

In [ ]:
print(data.shape)
print(data.columns)

As we can see in the command output we have 303 rows and 14 columns, and seeing the column names the value to predict could be in the 'output' column. 

In [ ]:
data['output'].unique()

If we show the different values in the ouput column we can see that we only have two distinct values, the 0 and the 1. This tells to us that we are in front of a binary classification problem.

# **Exploratory data analysis. (EDA)**

For now we know what type of problem we are faced, but we don't know nothing about the data quality, for this reaso, the first job is explore the data to get information about that. A good starting point is getting some general information about our dataset.

In [ ]:
data.info()

In the code above we can see that we have 303 entries and 14 columns (information extracted from the first two lines) but we already known this information. In the table we see the different column names, if has or not null values and his data type, and with this information we can make some assumptions like this: 
- We don't have null values, then we don't have to fill this white spaces.
- All columns are in a numeric type (integer and float), then we don't have to convert the content of these columns in to numeric type or modify the column type to numeric. 

In [ ]:
data.head()

In the table above we see the different columns and the content of the first five rows, and also we see that the range of the different values are different, e.g. the sex column contains values that are either 0 or 1 (depend on if the person is male or female) and the column named chol (cholesterol in mg/dL) contains high values, therefore we should standarize the data (modify the ranges of all the columns from -1 to 1) if we want to use algorithms that uses the distance to classify the data. If we would have different ranges in the columns the influence in the output of the columns will be different, this means that the model would take into account the columns with higher values to decide the final result. Before to standarize the model we should see if the columns have some correlation between them and decide if we should or not remove any column.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
sns.heatmap(data.corr(),cmap='coolwarm', annot=True)

Seeing the correlation between columns we can say that all the columns are necessary, or the same, we don't should remove any column because the correlations are near to 0. If we had two columns with a correlation around 1 or -1 we should remove one of this columns, because a high correlation can lead us to get bad results. 

Other thing to do is see if we have duplicated values, the duplicated values can lead us to a bad perform.

In [ ]:
data.duplicated().sum()

We have 1 duplicated value, this is not a problem because we can remove this value.

In [ ]:
data = data.drop_duplicates()
data.duplicated().sum()

Now we don't have duplicated values, the next step is know if we have either balanced or unbalanced data, in case that we don't have a balanced dataset we could have problems to predict the label with few samples. To comprove this we have to count the different entries with output equals to 1 and entries with output equals to 0.

In [ ]:
import seaborn as sns
sns.countplot(data['output']) 

We have around 140 samples of no heart attack and around 165 samples of heart attack, this dataset it is balanced because have around 50% for each sample to predict, in case that the dataset contains 70-30 or more, the dataset should considered unbalanced, and in this case the treatment of the data would be dfferent.

Other interesting thing to do is see if we have balanced data but not for the final result, but for sex column. This is a good practice, because we could have problems predict the heart attack for the sex with few samples.

In [ ]:
fig, ax3 = plt.subplots(figsize=(12,5))
grp2 = data.groupby(['sex', 'output'])['output'].count() #Rating mean
grp2.plot.bar(ax = ax3)

As we can see in the table, we have more samples of the sex 1 than the sex 0, this could be a problem.I do not know if the conditions for having an heart attack are different in men and women, for this reason i go to see the feature importance, if the importance of sex is low, I'm will drop this column, but this step will take place after train the model. 

# Training

Now that we have all the relevant information about our data set is time to make the necessary modifications and train the model. First of all we will select the independent variables (all the variables that helps us to predict the final result) and the dependent variable (variable to predict)

In [ ]:
x = data.iloc[:, 0:13].values #Independent variables
y = data.iloc[:, 13].values #Dependent variable

In [ ]:
#Data standarizaton
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)
x[1]

The data has been standarized, now we don't will have problems with the columns influence in the final result. As example we see the values for the different columns in the first row.

Now we need to split the data in a subset for train the model and a subset for test the performance of our model, for this reason we need to use the train_test_split function. Also we need to do this split for the independent variable and for the dependent variable.

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

Now it's time to train the model, but for do it we have a lot of models, and a lot of hyperparameters. If we don't know what model can be fitting better we can use the grid search technique. This technique helps us to train the model with more than one classifier (or one if we want) and his hyperparaments with his different values for each one. 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier 

pipe = Pipeline(steps = [('estimator', RandomForestClassifier())])

params = [{
    'estimator' : [SVC()],
    'estimator__C' : [0, 0.15, 0.2, 1, 10], # Larger C avoids more miss-classifications
    }, 
    {
    'estimator' : [DecisionTreeClassifier()], # 0.813 train, 0.815 sin tau3, 0.81 test
    'estimator__criterion' : ['gini', 'entropy'],
    'estimator__max_depth' : range(0, 10),  #Maximum depth of the tree
    'estimator__min_samples_split' : range(1,5), #Minimum of samples for split one node
    'estimator__min_samples_leaf': range(1,5) #Minimum of samples to be a leaf node
    }, 
    {
    'estimator' : [XGBClassifier()],  #0.8712 train y test, 0.8604 sin tau3
    'estimator__max_depth' : range(2,8), # range(4,8), #6 by default, then not remove
    'estimator__learning_rate' : [0.11, 0.12, 0.13], 
    'estimator__n_estimators' : range(100,106),
    'estimator__min_child_weight': range(1,4),
    'estimator__min_split_loss' : range(1,4) # One node will be splited if the loss function is equal or less than the parameter value (put in 0)
    },
    {
    'estimator' : [RandomForestClassifier()], #0.865 train, 0.866 sin tau3, 0.86 test
    'estimator__n_estimators' : range(297, 302), #Number of trees in the forest
    'estimator__max_depth' : range(3,6), 
    }]

model_grid = GridSearchCV(pipe, cv=5, param_grid = params, verbose=2)
model_grid.fit(x_train, y_train)

In [ ]:
model_grid.best_params_

We are trained different models, and the best model is Random Forest, the next step is evaluate the model with different metrics. The first metric that we should check is the accuracy, this metric evalueates the ability of the model for make a prediction. 

In [ ]:
print("Accuracy train: {:.3f}".format(model_grid.score(x_train, y_train))) #See the accuracy of training set
print("Accuracy test: {:.3f}".format(model_grid.score(x_test, y_test))) #Print the accuracy of test set

In this case we have an accuracy of 90% in the train set (the model is able to predict well the 90% of knowed samples) and is able to predict the 82'4% in the test set (data that the model has never seen) The accuracy are differents in both cases, this is quite normal but if we had a very high gap between the accuracies we have two cases: 
- The train model is very higher than the test model: In this case we talk about overffit, the model is able to do well predictions in date previously seen, but don't know how to generalize with data that never had seen.

- The model performs very poorly also in train set, in this case we talk about underfit. 

In our case we don't have any of this cases. 

In [ ]:
y_pred = model_grid.predict(x_test)

Another metric to predict is the confusion matrix, the diagonal of the matrix represents the correct predictions, the other values are wrong predicted values. 

In [ ]:
from sklearn.metrics import confusion_matrix
cnf = confusion_matrix(y_test, y_pred)
cnf

We have 34 negative cases labeled as negative cases, and 42 positives cases labeleds as positives, but then we have 11 cases labeleds as positives but actually are negatives, and 4 cases labeleds as negatives and are positives (in this cases this is a problem). Now let's analyze the ROC curve


In [ ]:
from sklearn import metrics
metrics.plot_roc_curve(model_grid, x_test, y_test)  
plt.show()

With the ROC curve we can obtain the proportion between true positives vs false positives, in our model we have a proportion of 0.92 or 92%

# Conclusions

We are trained a model to detect heart attacks, and we are achieve good results. This dataset are very easy to interpret because we don't have null values, unbalanced data, categorical variables and the amount of data is enough for do well predictions. In a real dataset we don't have this type of dataset and neither datasets with few columns with wich is easy to work. In real jobs we spend around the 80% of the time in EDA step, in our case we don't have spend to much time.

